___
# Ciência dos Dados - PROJETO 2

___
## Beni Stern

## Leonardo Costa

## Gabriel Duarte

___

___

## 1. Problema

O Classificador Naive-Bayes, o qual se baseia no uso do teorema de Bayes, é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser SPAM considerando as palavras em seu conteúdo e, de forma complementar, permite calcular a probabilidade de uma mensagem ser HAM dada as palavras descritas na mensagem.

Para realizar o MVP (minimum viable product) do projeto, você precisa programar uma versão do classificador que "aprende" o que é uma mensagem SPAM considerando uma base de treinamento e comparar o desempenho dos resultados com uma base de testes. 


___
## 2. Separação da base de dados em Treinamento e Teste

A base de dados deve ser separada em duas partes, aleatoriamente, considerando: 
    
    75% dos dados para a parte Treinamento; e
    25% dos dados para a parte Teste.

In [2]:
import pandas as pd
import os
import numpy as np

In [3]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Duarte\Downloads\INSPER aqui tbm\DADOS DP\Projeto2\Projeto-2-Filtro-Spam


In [4]:
dados = pd.read_excel("spamham2019.xlsx")

In [5]:
dados_aleatorio = dados.sample(frac=1)
dados_aleatorio = dados.sample(frac=1).reset_index(drop=True)
dados_treinamento = dados_aleatorio.loc[:4180:,:]
dados_teste = dados_aleatorio.loc[4180:,:]

In [25]:
dados_treinamento.head()

,Email,Class
0,Huh i cant thk of more oredi how many pages do...,ham
1,Wat r u doing now?,ham
2,Happy New year my dear brother. I really do mi...,ham
3,"Dear Voucher Holder, 2 claim this weeks offer,...",spam
4,Life is more strict than teacher... Bcoz Teach...,ham


In [7]:
dados_teste.head()

,Email,Class
4180,"Sorry, I'll call later",ham
4181,"Ya, i'm referin to mei's ex wat... No ah, wait...",ham
4182,K. I will sent it again,ham
4183,Especially since i talk about boston all up in...,ham
4184,"Sorry, I can't help you on this.",ham


___
## 3. Classificador Naive-Bayes

In [9]:
#def Filtro(base):
#    base.replace()
##    
#   
#    return base_limpa
#limpo = Filtro(dados_treinamento)

In [30]:
n_ham = dados_treinamento["Class"].value_counts()[0]
n_spam = dados_treinamento["Class"].value_counts()[1]
total = n_spam + n_ham

P_ham = n_ham/total
P_spam = n_spam/total

print("P(SPAM) = ", P_spam)
print("P(HAM)  = ", P_ham)

P(SPAM) =  0.13752690743841187
P(HAM)  =  0.8624730925615881


In [109]:
def P_WORD(palavra,baze):
    frase = baze["Email"]
    palavras_em_lista = []
    
    for linha in frase:
        lista_palavras = linha.split(" ")
        
        for p in lista_palavras:
            palavras_em_lista.append(p)        
    
    total_palavras = len(palavras_em_lista)
    palavras = pd.DataFrame(palavras_em_lista)
    contagem = palavras[0].value_counts()
    tabela_contagem = pd.DataFrame(contagem)
    n_palavra = tabela_contagem.loc[palavra,:][0]
    
    prob_palavra = n_palavra/total
    
    return prob_palavra

P_word = P_WORD("you",dados_treinamento)

P_wordSpam = (P_spam * P_word)/P_spam
P_wordHam = (P_ham * P_word)/P_ham

print("P(word|SPAM) = ", P_wordSpam)
print("P(word|HAM)  = ", P_wordHam)

P(word|SPAM) =  0.29131786653910546
P(word|HAM)  =  0.29131786653910546


In [ ]:
wgs84 4326

___
## 4. Qualidade do Classificador alterando a base de treinamento